In [1]:
!pip install flask-cors

  Using cached flask-3.0.3-py3-none-any.whl (101 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 KB 1.8 MB/s eta 0:00:00a 0:00:01
  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)


In [ ]:
!pip install -qq pyannote.audio==3.1.1
!pip install -qq ipython==7.34.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/117.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 50.0 MB/s eta 0:00:00
   ━━━━

In [ ]:
!pip install pyannote.database

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from huggingface_hub import login
login(token="Your-Token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
import pyannote
import numpy as np
import pytorch_lightning as pl
from pyannote.core import notebook, Segment
from pyannote.database import registry, FileFinder
from pyannote.audio import Model
from pyannote.audio import Inference
from pyannote.audio import Pipeline
from pyannote.audio.tasks import SpeakerDiarization
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.audio.utils.metric import DiscreteDiarizationErrorRate
from copy import deepcopy
from pyannote.audio.utils.signal import binarize
from pyannote.audio.pipelines.utils import get_devices
from pyannote.core import SlidingWindowFeature, Annotation
from pyannote.audio.tasks import SpeakerEmbedding
from pyannote.audio.tasks import Segmentation
from types import MethodType
from torch.optim import Adam
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.pipeline import Optimizer
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    RichProgressBar,
)

In [ ]:
!PYANNOTE_DATABASE_CONFIG="/content/drive/MyDrive/Pyannote2/Data/database.yml" pyannote-database info MyDatabase.SpeakerDiarization.MyProtocol

/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)
train
   48 files
   21h56m annotated
   21h56m of speech (100%)
   206 speakers
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will b

In [ ]:
from pyannote.database import registry, FileFinder

registry.load_database("/content/drive/MyDrive/Pyannote2/Data/database.yml")
dataset = registry.get_protocol("MyDatabase.SpeakerDiarization.MyProtocol", {"audio": FileFinder()})

In [ ]:
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.audio import Model
from pyannote.audio.utils.metric import DiarizationErrorRate
import torch

# 1. 학습된 모델 로드
finetuned_model_path = "/content/drive/MyDrive/Pyannote2/models/completed/finetuned_best_model.ckpt"
model = Model.from_pretrained(finetuned_model_path).to(torch.device("cuda"))

# 2. 최적화된 하이퍼파라미터 값 설정
best_segmentation_threshold = 0.6292053016753839
best_clustering_threshold = 0.7601635586431199

# 3. 학습된 모델 및 최적의 하이퍼파라미터로 파이프라인 구성
finetuned_pipeline = SpeakerDiarization(
    segmentation=model,
    clustering="AgglomerativeClustering"  # OracleClustering 대신 AgglomerativeClustering을 사용합니다.
).to(torch.device("cuda"))

# 4. 하이퍼파라미터 적용 (clustering 매개변수를 포함합니다)
finetuned_pipeline.instantiate({
    "segmentation": {
        "threshold": best_segmentation_threshold,
        "min_duration_off": 0.0
    },
    "clustering": {
        "method": "centroid",           # AgglomerativeClustering에서 사용 가능
        "min_cluster_size": 15,
        "threshold": best_clustering_threshold,
    },
})

# 5. 새로운 데이터에 대한 성능 평가
metric = DiarizationErrorRate()

for file in dataset.test():
    # 모델에 입력하여 결과 추론
    file["finetuned pipeline"] = finetuned_pipeline(file)
    # 성능 측정
    metric(file["annotation"], file["finetuned pipeline"], uem=file["annotated"])

print(f"The finetuned pipeline reaches a Diarization Error Rate (DER) of {100 * abs(metric):.1f}% on {dataset.name} test set.")


/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)


The finetuned pipeline reaches a Diarization Error Rate (DER) of 9.3% on MyDatabase.SpeakerDiarization.MyProtocol test set.


# 실제 배포시 함수

In [ ]:
import torch
from pyannote.audio import Pipeline
from pyannote.core import Annotation

# 미리 설정된 모델과 파라미터 로드
finetuned_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization").to(torch.device("cuda"))
finetuned_pipeline.instantiate({
    "segmentation": {
        "threshold": 0.6292053016753839,
        "min_duration_off": 0.52173913046875,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
        "threshold": 0.7601635586431199,
    },
})

def diarize_audio_file(audio_file_path: str) -> Annotation:
    # 단일 파일 설정
    single_file = {"uri": "unique_identifier", "audio": audio_file_path}

    # 파이프라인 적용
    diarization_result = finetuned_pipeline(single_file)

    return diarization_result
